#### This notebook is run inside a Databricks cluster. The following Tensorflow and Keras versions are required to correctly setup the sparkdl library. Although it is possible to install these packages locally, it is recommended to use the Databricks cluster. Specifically, we are using the 6.5 ML runtime and r4.2xlarge worker type (61 GB memory, 8 cores, 2 DBU). Databricks provides a 2-week free trial for the non-community version.

## NOTE: Some features might not display correctly inside Jupyter Notebook since this notebook was created inside Databricks cluster. Hence, it is advisable to run it inside a Databricks cluster. Please refer to the html file for correct display of the output.

In [2]:
import tensorflow
import keras
print(tensorflow.__version__)
print(keras.__version__)

1.15.0
2.2.5

### Load Data after placing it on the cluster

In [4]:
img_dir = '/tmp/corona/corona/corona'
image_df = spark.read.format("image").load(img_dir+'/train')
image_df.show()

+--------------------+
 image|
+--------------------+
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
[dbfs:/tmp/corona...|
+--------------------+
only showing top 20 rows

In [5]:
display(image_df)

image


### Preprocess Data and Split into Train and Test Sets

In [7]:
import pyspark.sql.functions as F
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType

data_path = "/tmp/corona/metadata.csv"
meta_data = spark.read.format('csv').options(header='true', inferschema='true').load(data_path)

meta_data_with_labels = meta_data.withColumn(
    'new_label',
    F.when((F.col("Label")=="Normal"), 0.0).otherwise(1.0)
)

number_of_samples = meta_data_with_labels.count()
train_set = meta_data_with_labels.where(meta_data.Dataset_type == "TRAIN")
test_set = meta_data_with_labels.where(meta_data.Dataset_type == "TEST")
num_train = train_set.count()
num_test = test_set.count()

train_set = train_set.select('X_ray_image_name', 'new_label').withColumnRenamed('X_ray_image_name', 'uri').withColumnRenamed('new_label', 'label')
test_set = test_set.select('X_ray_image_name', 'new_label').withColumnRenamed('X_ray_image_name', 'uri').withColumnRenamed('new_label', 'label')

@udf(returnType=StringType())
def get_absolute_path_train(img_file):
    abs_path = "/dbfs/tmp/corona/corona/corona/train/"+str(img_file)
    return abs_path
train_set = train_set.withColumn("uri", get_absolute_path_train(F.col("uri")))

@udf(returnType=StringType())
def get_absolute_path_test(img_file):
    abs_path = "/dbfs/tmp/corona/corona/corona/test/"+str(img_file)
    return abs_path
test_set = test_set.withColumn("uri", get_absolute_path_test(F.col("uri")))

train_set.show(3)
train_set.printSchema()

test_set.show(3)
test_set.printSchema()

+--------------------+-----+
 uri|label|
+--------------------+-----+
/dbfs/tmp/corona/...| 0.0|
/dbfs/tmp/corona/...| 0.0|
/dbfs/tmp/corona/...| 0.0|
+--------------------+-----+
only showing top 3 rows

root
-- uri: string (nullable = true)
-- label: double (nullable = false)

+--------------------+-----+
 uri|label|
+--------------------+-----+
/dbfs/tmp/corona/...| 0.0|
/dbfs/tmp/corona/...| 0.0|
/dbfs/tmp/corona/...| 0.0|
+--------------------+-----+
only showing top 3 rows

root
-- uri: string (nullable = true)
-- label: double (nullable = false)

### Prepare Data for Input into Custom Model

In [9]:
import pyspark.ml.linalg as spla
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.linalg import Vectors, VectorUDT

@udf(VectorUDT())
def get_ohe(label):
  if label == 0.0:
      ohe = [1.0, 0.0]
  elif label == 1.0:
      ohe = [0.0, 1.0]
  ohe = spla.Vectors.dense(ohe)
  return ohe

train_set_custom_model = train_set.withColumn("ohe", get_ohe(F.col("label")))
train_set_custom_model.show(3)
train_set_custom_model.printSchema()

test_set_custom_model = test_set.withColumn("ohe", get_ohe(F.col("label")))
test_set_custom_model.show(3)
test_set_custom_model.printSchema()

+--------------------+-----+---------+
 uri|label| ohe|
+--------------------+-----+---------+
/dbfs/tmp/corona/...| 0.0|[1.0,0.0]|
/dbfs/tmp/corona/...| 0.0|[1.0,0.0]|
/dbfs/tmp/corona/...| 0.0|[1.0,0.0]|
+--------------------+-----+---------+
only showing top 3 rows

root
-- uri: string (nullable = true)
-- label: double (nullable = false)
-- ohe: vector (nullable = true)

+--------------------+-----+---------+
 uri|label| ohe|
+--------------------+-----+---------+
/dbfs/tmp/corona/...| 0.0|[1.0,0.0]|
/dbfs/tmp/corona/...| 0.0|[1.0,0.0]|
/dbfs/tmp/corona/...| 0.0|[1.0,0.0]|
+--------------------+-----+---------+
only showing top 3 rows

root
-- uri: string (nullable = true)
-- label: double (nullable = false)
-- ohe: vector (nullable = true)

### Create and Train Custom Model

In [11]:
from keras.layers import Activation, Dense, Flatten
from keras.models import Sequential
import PIL.Image
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from sparkdl.estimators.keras_image_file_estimator import KerasImageFileEstimator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

model = Sequential()
model.add(Flatten(input_shape=(299, 299, 3)))
model.add(Dense(2))
model.add(Activation("softmax"))
model.save('/tmp/corona_model.h5')  # saves to the local filesystem
dbfs_corona_model_path = 'dbfs:/models/corona_model.h5'
dbutils.fs.cp('file:/tmp/corona_model.h5', dbfs_corona_model_path) 

def load_image_from_uri(local_uri):
  img = (PIL.Image.open(local_uri).convert('RGB').resize((299, 299), PIL.Image.ANTIALIAS))
  img_arr = np.array(img).astype(np.float32)
  img_tnsr = preprocess_input(img_arr[np.newaxis, :])
  print(img_tnsr.shape)
  return img_tnsr


dbutils.fs.cp(dbfs_corona_model_path, 'file:/tmp/corona_model.h5')
estimator = KerasImageFileEstimator(inputCol="uri",
                                    outputCol="prediction",
                                    labelCol="ohe",
                                    imageLoader=load_image_from_uri,
                                    kerasOptimizer='adam',
                                    kerasLoss='categorical_crossentropy',
                                    modelFile='/tmp/corona_model.h5')

/local_disk0/tmp/1589483106863-0/PythonShell.py:33: DeprecationWarning: Call to deprecated class KerasImageFileEstimator. (KerasImageFileEstimator will be removed in the next major Databricks Runtime ML release. To replace a KerasImageFileEstimator workflow, please use Distributed Hyperopt with SparkTrials to distribute model tuning.)
 import IPython

### Hyperparameter Tuning

In [13]:
paramGrid = (
  ParamGridBuilder()
  .addGrid(estimator.kerasFitParams, [{"batch_size": 2, "verbose": 0},
                                      {"batch_size": 2, "verbose": 0}])
  .build()
)
ev = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label" )
cross_validator = CrossValidator(estimator=estimator, estimatorParamMaps=paramGrid, evaluator=ev, numFolds=2)

### Train

In [15]:
custom_model = cross_validator.fit(train_set_custom_model)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/estimators/keras_image_file_estimator.py:250: DeprecationWarning: Call to deprecated class KerasImageFileTransformer. (KerasImageFileTransformer will be removed in the next major Databricks Runtime ML release. Please use binary file data source and Pandas UDF instead.)
 yield i, self._copyValues(KerasImageFileTransformer(modelFile=model_filename),
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/estimators/keras_image_file_estimator.py:250: DeprecationWarning: Call to deprecated class KerasImageFileTransformer. (KerasImageFileTransformer will be removed in the next major Databricks Runtime ML release. Please use binary file data source and Pandas UDF instead.)
 yield i, self._copyValues(KerasImageFileTransformer(modelFile=model_filename),
/databricks/python/lib/python3.7/copyreg.py:88: DeprecationWarning: Call to deprecated class KerasImageFileTransformer. (KerasImageFileTransformer will be removed in the next major Databricks Runtime ML release. Please use binary file data source and Pandas UDF instead.)
 return cls.__new__(cls, *args)
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/param/shared_params.py:170: DeprecationWarning: Call to deprecated function (or staticmethod) strip_and_freeze_until. (strip_and_freeze_until() will be removed in next major Databricks Runtime ML release. Please use Pandas UDF for distributed model inference.)
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/transformers/keras_image.py:71: DeprecationWarning: Call to deprecated class TFImageTransformer. (TFImageTransformer will be removed in the next major Databricks Runtime ML release. Please use Pandas UDF for distributed model inference.)
 outputMode=self.getOrDefault(self.outputMode))
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/transformers/tf_image.py:216: DeprecationWarning: Call to deprecated function (or staticmethod) strip_and_freeze_until. (strip_and_freeze_until() will be removed in next major Databricks Runtime ML release. Please use Pandas UDF for distributed model inference.)
 gdef = tfx.strip_and_freeze_until([self._getFinalOutputOpName()], tf_graph)
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.
/databricks/python/lib/python3.7/copyreg.py:88: DeprecationWarning: Call to deprecated class KerasImageFileTransformer. (KerasImageFileTransformer will be removed in the next major Databricks Runtime ML release. Please use binary file data source and Pandas UDF instead.)
 return cls.__new__(cls, *args)
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/param/shared_params.py:170: DeprecationWarning: Call to deprecated function (or staticmethod) strip_and_freeze_until. (strip_and_freeze_until() will be removed in next major Databricks Runtime ML release. Please use Pandas UDF for distributed model inference.)
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/transformers/keras_image.py:71: DeprecationWarning: Call to deprecated class TFImageTransformer. (TFImageTransformer will be removed in the next major Databr

### Test

In [17]:
ev.evaluate(custom_model.transform(test_set_custom_model)

/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/param/shared_params.py:170: DeprecationWarning: Call to deprecated function (or staticmethod) strip_and_freeze_until. (strip_and_freeze_until() will be removed in next major Databricks Runtime ML release. Please use Pandas UDF for distributed model inference.)
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/transformers/keras_image.py:71: DeprecationWarning: Call to deprecated class TFImageTransformer. (TFImageTransformer will be removed in the next major Databricks Runtime ML release. Please use Pandas UDF for distributed model inference.)
 outputMode=self.getOrDefault(self.outputMode))
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/transformers/tf_image.py:216: DeprecationWarning: Call to deprecated function (or staticmethod) strip_and_freeze_until. (strip_and_freeze_until() will be removed in next major Databricks Runtime ML release. Please use Pandas UDF for distributed model inference.)
 gdef = tfx.strip_and_freeze_until([self._getFinalOutputOpName()], tf_graph)
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.
Out[101]: 0.39070189070189076

### Transfer Learning

### Load Dataframe with Ground Truth Labels and Image Names

In [20]:
comparison_df1 = meta_data_with_labels.select('X_ray_image_name', 'new_label').withColumnRenamed('X_ray_image_name', 'key').withColumnRenamed('new_label', 'label')
comparison_df1.show(5)
print(comparison_df1.count())
print("Pneumonia patients:")
print(comparison_df1.filter("label == 1.0").count())
print("Healthy patients:")
print(comparison_df1.filter("label == 0.0").count())

+-----------------+-----+
 key|label|
+-----------------+-----+
IM-0128-0001.jpeg| 0.0|
IM-0127-0001.jpeg| 0.0|
IM-0125-0001.jpeg| 0.0|
IM-0122-0001.jpeg| 0.0|
IM-0119-0001.jpeg| 0.0|
+-----------------+-----+
only showing top 5 rows

5910
Pneumonia patients:
4334
Healthy patients:
1576

### Load Train and Test Sets as Image Schemas

In [22]:
from pyspark.sql.functions import lit
from sparkdl.image import imageIO

img_dir = "tmp/corona/corona/corona/"
train_df = spark.read.format("image").load(img_dir + "/train")

@udf(returnType=StringType())
def get_key(img_file):
    key = img_file.split('/')[-1]
    return key
train_df = train_df.withColumn("key", get_key(F.col("image.origin")))
train_df.show(3)

test_df = spark.read.format("image").load(img_dir + "/test")

@udf(returnType=StringType())
def get_key(img_file):
    key = img_file.split('/')[-1]
    return key
test_df = test_df.withColumn("key", get_key(F.col("image.origin")))
test_df.show(3)

print(train_df.count())
print(test_df.count())

# repartition train and test dfs so that each partition has a small size
train_df = train_df.repartition(100)
test_df = test_df.repartition(100)

+--------------------+--------------------+
 image| key|
+--------------------+--------------------+
[dbfs:/tmp/corona...|2C10A413-AABE-480...|
[dbfs:/tmp/corona...|E1724330-1866-458...|
[dbfs:/tmp/corona...|7E335538-2F86-424...|
+--------------------+--------------------+
only showing top 3 rows

+--------------------+--------------------+
 image| key|
+--------------------+--------------------+
[dbfs:/tmp/corona...| IM-0045-0001.jpeg|
[dbfs:/tmp/corona...| IM-0025-0001.jpeg|
[dbfs:/tmp/corona...|NORMAL2-IM-0282-0...|
+--------------------+--------------------+
only showing top 3 rows

5309
624

### Create Train and Test DFs for Input into Transfer Learning Model

In [24]:
train_df = train_df.join(comparison_df1, ["key"])
train_df = train_df.select('image', 'label')
train_df.show(3)
print(train_df.count())

test_df = test_df.join(comparison_df1, ["key"])
test_df = test_df.select('image', 'label')
test_df.show(3)
print(test_df.count())

+--------------------+-----+
 image|label|
+--------------------+-----+
[dbfs:/tmp/corona...| 0.0|
[dbfs:/tmp/corona...| 0.0|
[dbfs:/tmp/corona...| 0.0|
+--------------------+-----+
only showing top 3 rows

5286
+--------------------+-----+
 image|label|
+--------------------+-----+
[dbfs:/tmp/corona...| 0.0|
[dbfs:/tmp/corona...| 0.0|
[dbfs:/tmp/corona...| 1.0|
+--------------------+-----+
only showing top 3 rows

624

In [25]:
print(f"Pneumonia patients in train set = {train_df.filter('label == 1.0').count()}")
print(f"Pneumonia patients in test set = {test_df.filter('label == 1.0').count()}")

Pneumonia patients in train set = 3944
Pneumonia patients in test set = 390

### Transfer Learning + Logistic Regression with 5 Different Pretrained Models

In [27]:
import warnings
warnings.filterwarnings("ignore")
sc.setLogLevel("ERROR")

from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

models = ['InceptionV3', 'Xception', 'ResNet50', 'VGG16', 'VGG19']
for model in models:
  print(f"Model = {model}")
  featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName=model)
  lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
  p = Pipeline(stages=[featurizer, lr])
  p_model = p.fit(train_df)
  
  # Evaluate on the test set
  tested_df = p_model.transform(test_df)
  evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
  print("Test set accuracy = " + str(evaluator.evaluate(tested_df.select("prediction", "label"))))

Model = InceptionV3
/local_disk0/tmp/1589483106863-0/PythonShell.py:13: DeprecationWarning: Call to deprecated class DeepImageFeaturizer. (DeepImageFeaturizer will be removed in the next major Databricks Runtime ML release. Please use Pandas UDF instead.)
 import sys
Test set accuracy = 0.782051282051282
Model = Xception
/local_disk0/tmp/1589483106863-0/PythonShell.py:13: DeprecationWarning: Call to deprecated class DeepImageFeaturizer. (DeepImageFeaturizer will be removed in the next major Databricks Runtime ML release. Please use Pandas UDF instead.)
 import sys
Test set accuracy = 0.7884615384615384
Model = ResNet50
/local_disk0/tmp/1589483106863-0/PythonShell.py:13: DeprecationWarning: Call to deprecated class DeepImageFeaturizer. (DeepImageFeaturizer will be removed in the next major Databricks Runtime ML release. Please use Pandas UDF instead.)
 import sys
Test set accuracy = 0.7548076923076923
Model = VGG16
/local_disk0/tmp/1589483106863-0/PythonShell.py:13: DeprecationWarning: Call to deprecated class DeepImageFeaturizer. (DeepImageFeaturizer will be removed in the next major Databricks Runtime ML release. Please use Pandas UDF instead.)
 import sys
Test set accuracy = 0.7483974358974359
Model = VGG19
/local_disk0/tmp/1589483106863-0/PythonShell.py:13: DeprecationWarning: Call to deprecated class DeepImageFeaturizer. (DeepImageFeaturizer will be removed in the next major Databricks Runtime ML release. Please use Pandas UDF instead.)
 import sys
Test set accuracy = 0.7612179487179487

### Transfer Learning + Gradient Boosted Trees with 5 Different Pretrained Models

In [29]:
import warnings
warnings.filterwarnings("ignore")
sc.setLogLevel("ERROR")

from pyspark.ml.classification import LogisticRegression, GBTClassifier
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


models = ['InceptionV3', 'Xception', 'ResNet50', 'VGG16', 'VGG19']
for model in models:
  print(f"Model = {model}")
  featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName=model)
  gbt = GBTClassifier(maxIter=5, labelCol="label")
  p = Pipeline(stages=[featurizer, gbt])
  p_model = p.fit(train_df)
  
  # Evaluate on the test set
  tested_df = p_model.transform(test_df)
  evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
  print("Test set accuracy = " + str(evaluator.evaluate(tested_df.select("prediction", "label"))))

### Analyze the Predictions to Check Incorrectly Classified Examples

In [31]:
tested_df.show(3)

+--------------------+-----+--------------------+--------------------+--------------------+----------+
 image|label| features| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+----------+
[dbfs:/tmp/corona...| 1.0|[0.0,0.0,0.0,3.75...|[-1.6480436540101...|[0.16137353065405...| 1.0|
[dbfs:/tmp/corona...| 1.0|[0.0,0.0,0.0,0.0,...|[-4.3143478761144...|[0.01319873237939...| 1.0|
[dbfs:/tmp/corona...| 0.0|[0.0,0.0,0.0,0.0,...|[-2.5372782563018...|[0.07328580608911...| 1.0|
+--------------------+-----+--------------------+--------------------+--------------------+----------+
only showing top 3 rows

In [32]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import expr
def _prob(v):
  return float(v.array[1])
prob = udf(_prob, DoubleType())

incorrect_df = tested_df.withColumn("prob", prob(tested_df.probability))
incorrect_df = incorrect_df.orderBy(expr("abs(prob - label)"), ascending=False)
display(incorrect_df.select("image.origin", "prob", "label").limit(10))

origin,prob,label
dbfs:/tmp/corona/corona/corona/test/NORMAL2-IM-0256-0001.jpeg,0.9958329769242518,0.0
dbfs:/tmp/corona/corona/corona/test/NORMAL2-IM-0206-0001.jpeg,0.9884361140629199,0.0
dbfs:/tmp/corona/corona/corona/test/NORMAL2-IM-0232-0001.jpeg,0.986108476341519,0.0
dbfs:/tmp/corona/corona/corona/test/NORMAL2-IM-0246-0001.jpeg,0.9838532152041709,0.0
dbfs:/tmp/corona/corona/corona/test/NORMAL2-IM-0251-0001.jpeg,0.9767599907071957,0.0
dbfs:/tmp/corona/corona/corona/test/IM-0073-0001.jpeg,0.9746315928570622,0.0
dbfs:/tmp/corona/corona/corona/test/NORMAL2-IM-0349-0001.jpeg,0.9744397260067932,0.0
dbfs:/tmp/corona/corona/corona/test/NORMAL2-IM-0221-0001.jpeg,0.9727955156073012,0.0
dbfs:/tmp/corona/corona/corona/test/NORMAL2-IM-0330-0001.jpeg,0.9716742100531509,0.0
dbfs:/tmp/corona/corona/corona/test/NORMAL2-IM-0233-0001.jpeg,0.969046532559076,0.0


### Deploy the Custom Model as Spark SQL UDF

In [34]:
registerKerasImageUDF("corona_model_udf", "/tmp/corona_model.h5") # register model as a Spark SQL UDF
df = spark.read.format("image").load(img_dir + "/test").limit(1) # load image data as a column formatted as an image struct ImageSchema
df.registerTempTable("sample_images") # register table
# %sql
# SELECT corona_model_udf(image) as predictions from sample_images

/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/graph/builder.py:90: DeprecationWarning: Call to deprecated function (or staticmethod) strip_and_freeze_until. (strip_and_freeze_until() will be removed in next major Databricks Runtime ML release. Please use Pandas UDF for distributed model inference.)
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/graph/builder.py:90: DeprecationWarning: Call to deprecated function (or staticmethod) strip_and_freeze_until. (strip_and_freeze_until() will be removed in next major Databricks Runtime ML release. Please use Pandas UDF for distributed model inference.)
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/graph/builder.py:90: DeprecationWarning: Call to deprecated function (or staticmethod) strip_and_freeze_until. (strip_and_freeze_until() will be removed in next major Databricks Runtime ML release. Please use Pandas UDF for distributed model inference.)
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.
/databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.6.0-db1-spark2.4_shaded.jar/sparkdl/graph/builder.py:90: DeprecationWarning: Call to deprecated function (or staticmethod) strip_and_freeze_until. (strip_and_freeze_until() will be removed in next major Databricks Runtime ML release. Please use Pandas UDF for distributed model inference.)
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.